In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import csv
from datetime import timedelta

In [3]:
def MapAnything(Dict,Name):
    for k in Dict.keys():
        for ele in Dict[k]:
            if str.strip(str.lower(str(ele))) == str.strip(str.lower(str(Name))):
                return k
    return ''
def findCombinationFuelAndPlant(CommonParameters,Fuel,Plant):
    
    CommonParameters['Technology'] = [str.strip(str.lower(str(item))) for item in CommonParameters['Technology']]
    CommonParameters['Fuel'] = [str.strip(str.lower(str(item))) for item in CommonParameters['Fuel']]
    
    Selection = np.logical_and(CommonParameters['Technology'] == str.strip(str.lower(str(Plant))),
                               CommonParameters['Fuel'] == str.strip(str.lower(str(Fuel))))
    Selected = CommonParameters[Selection]
    return pd.DataFrame(data = Selected.values, columns = Selected.columns)

def findCombinationFuelAndPlant2(CommonParameters,Fuel,Plant):
    
    CommonParameters['Technology'] = [str.strip(str.lower(str(item))) for item in CommonParameters['Technology']]
    CommonParameters['Fuel 1'] = [str.strip(str.lower(str(item))) for item in CommonParameters['Fuel 1']]
    
    Selection = np.logical_and(CommonParameters['Technology'] == str.strip(str.lower(str(Plant))),
                               CommonParameters['Fuel 1'] == str.strip(str.lower(str(Fuel))))
    Selected = CommonParameters[Selection]
    return pd.DataFrame(data = Selected.values, columns = Selected.columns)

# 1- Power Demand Data

In [10]:
Saudi_zones = ['SA_EOA','SA_HAIL','SA_NEOA','SA_NWOA','SA_QASSIM','SA_RIYADH','SA_SOA','SA_WOA']
GCC_zones = ['BA','KW','OM','QA','UAE']
RegionNames = GCC_zones + Saudi_zones

In [11]:
RegionNamesDict = {
    'BA':['BAH'],
    'KW':['KUW'],
    'OM':['OMA'],
    'QA':['QAT'],
    'SA_EOA':['EOA'],
    'SA_HAIL':['HAIL'],
    'SA_NEOA':['NEOA'],
    'SA_NWOA':['NWOA'],
    'SA_QASSIM':['QASSIM'],
    'SA_RIYADH':['RIYADH','COA'],
    'SA_SOA':['SOA'],
    'SA_WOA':['WOA'],
    'UAE':['UAE']
}

In [12]:
Origin = pd.read_excel('../Database/RawData/Demand_MW.xlsx')
#Origin_GCC = pd.read_excel('../Database/RawData/GCCIA Demand_MW.xlsx')

### Create Date Time list in required format

In [14]:
#Read from file and generate dates
Dates = Origin.values[18:21,1:]
AllDates = []
for i in range(np.shape(Dates)[1]):
    Hour = Dates[:,i][0] - 1
    YearMonthDay = str.strip(str(Dates[:,i][2]))
    NewDate = datetime.strptime(YearMonthDay + ' ' + str(Hour),'%d/%m/%Y %H')
    AllDates.append(NewDate)

In [20]:
#Generate Own dates
Start = AllDates[0]
NewAllDates = []
NewAllDates.append(Start)
for item in range(8783):
    Start = Start + timedelta(seconds=3600)
    NewAllDates.append(Start)
AllDates = NewAllDates

### Create demand data for each zone

In [32]:
RegionNames = GCC_zones + Saudi_zones
for reg in RegionNames:
    AllMappings = RegionNamesDict[reg]
    Selection = False
    for name in AllMappings:
        Selection = np.logical_or(Selection,Origin.values[:,0] == name)
        Selected = Origin.values[Selection]
    RegionData = pd.DataFrame(np.zeros([len(AllDates),2]))
    RegionData[1] = Selected[0][1:]
    RegionData[0] = AllDates
    
    if not os.path.exists('../Database/Load_RealTime/'+reg):
        os.makedirs('../Database/Load_RealTime/'+reg)
    
    if not os.path.exists('../Database/Load_RealTime/'+reg+'/1h'):
        os.makedirs('../Database/Load_RealTime/'+reg+'/1h')
    
    RegionData.to_csv('../Database/Load_RealTime/'+reg+'/1h/2016.csv',index=False,
                 date_format = '%Y-%m-%d %H:%M:%S+%S:%S',header=False)
    print 'Zone '+reg+' has been written'

Zone BA has been written
Zone KW has been written
Zone OM has been written
Zone QA has been written
Zone UAE has been written
Zone SA_EOA has been written
Zone SA_HAIL has been written
Zone SA_NEOA has been written
Zone SA_NWOA has been written
Zone SA_QASSIM has been written
Zone SA_RIYADH has been written
Zone SA_SOA has been written
Zone SA_WOA has been written


# 2- Power Plants Data

## - Create Common Parameters (average) from EU data

In [9]:
#The imported file must have two columns with header "Technology" & "Fuel"
EU_PowerPlants = pd.read_excel('../Database/RawData/All_EU_PowerPlants.xlsx')

In [10]:
Technology_Column = EU_PowerPlants.columns.get_loc("Technology")
Fuel_Column = EU_PowerPlants.columns.get_loc("Fuel")

#Append a list of indices for columns with numbers (not string)
Integer_Indices = []
for i, ele in enumerate(range(np.shape(EU_PowerPlants)[1])):
    try:
            np.mean(EU_PowerPlants.values[:,ele].astype(float))
            Integer_Indices.append(ele)
    except Exception as e:
            print e.args
            pass      
'''
for i, ele in enumerate(EU_PowerPlants.values[0]):
    try:
        np.float(ele)
        Integer_Indices.append(i)
    except Exception as e:
        print e.args
        pass
'''

AllCombs = []
for comb in EU_PowerPlants.drop_duplicates(['Technology','Fuel']).values:
    Selection = np.logical_and(EU_PowerPlants['Technology'] == comb[Technology_Column],EU_PowerPlants['Fuel'] == comb[Fuel_Column])
    Row = comb
    X = EU_PowerPlants[Selection].values
    
    #Remove columns with all 'nans' because in this case np.nanmean gives error
    Integer_Indices_nonan = []
    for colum_num in Integer_Indices:
        if np.sum(np.isnan(X[:,colum_num].astype(float)) == False) > 0:
            Integer_Indices_nonan.append(colum_num)
    Row[Integer_Indices_nonan] = np.nanmean(EU_PowerPlants[Selection].values[:,Integer_Indices_nonan],axis = 0)        
            
    AllCombs.append(Row)

('could not convert string to float: DispaSET FossilGas_NL',)
('could not convert string to float: NL',)
('could not convert string to float: COMC',)
('could not convert string to float: GAS',)


In [11]:
CommonParametrs_EU = pd.DataFrame(AllCombs,columns=EU_PowerPlants.keys())
EU_columns = ['Technology','Fuel','Efficiency','MinUpTime','MinDownTime','RampUpRate','RampDownRate',
                    'StartUpCost','NoLoadCost','RampingCost','PartLoadMin','MinEfficiency','StartUpTime','CO2Intensity','CHPType']
CommonParametrs_EU[EU_columns]

,Technology,Fuel,Efficiency,MinUpTime,MinDownTime,RampUpRate,RampDownRate,StartUpCost,NoLoadCost,RampingCost,PartLoadMin,MinEfficiency,StartUpTime,CO2Intensity,CHPType
0,COMC,GAS,0.523159,4.615385,1.188811,0.035847,0.036577,102145.407507,340.063748,0,0.410505,0.512154,0.874126,80.074341,0.45
1,HPHS,WAT,0.865333,0.000000,0.000000,1.018893,1.018893,0.000000,0.000000,0,0.000000,0.843111,0.284444,2400.000000,0.00
2,STUR,HRD,0.388409,7.787879,5.803030,0.016492,0.016735,114088.100966,68.292211,0,0.304903,0.387803,6.257576,23.614629,0.95
3,HDAM,WAT,0.883333,0.000000,0.000000,0.634963,0.634963,0.000000,0.000000,0,0.000000,0.772222,0.455556,2400.000000,0.00
4,HROR,WAT,1.000000,0.000000,0.000000,1.050024,1.137491,25.120749,0.000000,0,0.000000,0.946970,0.306061,5600.000000,0.00
5,PHOT,SUN,1.000000,0.000000,0.000000,0.548703,0.548703,0.000000,0.000000,0,0.000000,0.937500,0.125000,6750.000000,0.00
6,WTON,WIN,1.000000,0.000000,0.000000,0.607120,0.607120,0.000000,0.000000,0,0.000000,0.958333,0.083333,675.000000,0.00
7,STUR,BIO,0.407778,5.888889,4.333333,0.014506,0.014578,7796.375710,449.237262,0,0.389523,0.396111,2.444444,0.413474,NaN
8,STUR,OIL,0.376502,7.058824,5.352941,0.022963,0.024596,29508.530032,186.160372,0,0.337731,0.362628,4.395349,139.809801,0.79
9,STUR,NUC,0.405000,24.000000,24.000000,0.003552,0.003552,200000.000000,0.000000,0,0.900000,0.405000,64.615385,153.030769,0.00


### Make some changes with assumptions

In [12]:
# Chagnge CHPType values from float numbers to Nan's 
# (CHPType unit should be: extraction/back-pressure/other NOT numbers)
CommonParametrs_EU['CHPType'] = np.nan
CommonParametrs_EU = CommonParametrs_EU[CommonParametrs_EU['Technology'] != 'Other']

# set the 'NoLoadCost' for ICEN & OIL to be equal to 'NoLoadCost' for GTUR & OIL
CommonParametrs_EU['NoLoadCost'][12] = CommonParametrs_EU['NoLoadCost'][11]

# set the 'NoLoadCost' for COMC & OIL to be equal to 'NoLoadCost' for COMC & GAS
CommonParametrs_EU['NoLoadCost'][13] = CommonParametrs_EU['NoLoadCost'][0]

# set the 'StartUpTime' for WTOF & WIN to be equal to 'StartUpTime' for WTON & WIN
CommonParametrs_EU['StartUpTime'][10] = CommonParametrs_EU['StartUpTime'][6]

# set the 'StartUpTime' for GTUR & OIL to be equal to 'StartUpTime' for GTUR & GAS
CommonParametrs_EU['StartUpTime'][11] = CommonParametrs_EU['StartUpTime'][16]

# set the 'StartUpTime' for ICEN & OIL to be equal to half the 'StartUpTime' for GTUR & GAS
CommonParametrs_EU['StartUpTime'][12] = CommonParametrs_EU['StartUpTime'][16]*0.5

# set the 'StartUpTime' for COMC & OIL to be equal to 'StartUpTime' for COMC & GAS
CommonParametrs_EU['StartUpTime'][13] = CommonParametrs_EU['StartUpTime'][0]

# set the 'StartUpTime' for STUR & GAS to be equal to 'StartUpTime' for STUR & OIL
CommonParametrs_EU['StartUpTime'][18] = CommonParametrs_EU['StartUpTime'][8]

# set the 'StartUpTime' for STUR & WST to be equal to 'StartUpTime' for STUR & BIO
CommonParametrs_EU['StartUpTime'][15] = CommonParametrs_EU['StartUpTime'][7]

# Add other combinations and set values to be equal to somehow a similar combination
# combination: GTUR & OIL, ICEN & OIL, STUR & OIL, STUR & BIO, STUR & WST
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['GTUR','DSL']+list(CommonParametrs_EU[EU_columns].loc[11]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['ICEN','GAS']+list(CommonParametrs_EU[EU_columns].loc[12]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['ICEN','DSL']+list(CommonParametrs_EU[EU_columns].loc[12]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['STUR','HFO']+list(CommonParametrs_EU[EU_columns].loc[8]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['STUR','MSW']+list(CommonParametrs_EU[EU_columns].loc[7]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['LFGG','LFG']+list(CommonParametrs_EU[EU_columns].loc[15]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['CPV','SUN']+list(CommonParametrs_EU[EU_columns].loc[5]['Efficiency':].values)], columns=EU_columns),ignore_index=True)
CommonParametrs_EU = CommonParametrs_EU.append(pd.DataFrame([['CSP','SUN']+list(CommonParametrs_EU[EU_columns].loc[5]['Efficiency':].values)], columns=EU_columns),ignore_index=True)

c:\python27\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
c:\python27\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
c:\python27\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\python27\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set

#### -- Changes on CO2 emissions acourding to EPA Emission Factors 

$$ CO2\,emissions\, [\frac{Tonne\,CO2}{MWh}]  = CO2\,emissions\, [\frac{kg\,CO2}{MMBTU}] × [\frac{1\,Tonne}{1000\,kg}] ×  Heat\,Rate\,\, [\frac{MMBTU}{MWh}] $$

$$  Heat\,Rate\,\, [\frac{MMBTU}{MWh}] = \frac{[\frac{3.41214163\,MMBTU}{1 MWh}]}{Efficiency\,\, η\,\, (\%)} $$

Side Note:
$$  CO2\,emissions\, [\frac{Tonne\,CO2}{MWh}]  = CO2\,emissions\, [\frac{kg\,CO2}{short\,ton}] × [\frac{1\,Tonne}{1000\,kg}] ×  Heat\,Rate\,\, [\frac{MMBTU}{MWh}] × (Higher\,Heating\,Value\,(HHV))^{-1}[\frac{short\,ton}{MMBTU}]  $$

In [13]:
CO2_emissions = pd.read_excel('../Database/RawData/Fuels CO2 emissions & Heat contents.xlsx')
CO2_emissions

,Fuel,CO2 factor (kg CO2 per mmBtu),CO2 factor (kg CO2 per scf),CO2 factor (kg CO2 per gallon),CO2 factor (kg CO2 per short ton),HHV (mmBtu per scf),HHV (mmBtu per gallon),HHV (mmBtu per short ton)
0,Natural Gas,53.06,0.054440,NaN,NaN,0.001026,NaN,NaN
1,Landfill Gas,52.07,0.025254,NaN,NaN,0.000485,NaN,NaN
2,Coal and coke - Mixed (Electric Power Sector),95.52,NaN,NaN,1885.0,NaN,NaN,19.73
3,Municipal Solid Waste,90.70,NaN,NaN,902.0,NaN,NaN,9.95
4,Crude Oil,74.54,NaN,10.29,NaN,NaN,0.138,NaN
5,HFO (Residual Fuel Oil No. 6),75.10,NaN,11.27,NaN,NaN,0.150,NaN
6,Diesel Fuel (Distillate Fuel Oil No. 2),73.96,NaN,10.21,NaN,NaN,0.138,NaN


In [14]:
# Changing the CO2 emissions factors according to EPA Emission Factors 
# (https://www.epa.gov/sites/production/files/2018-03/documents/emission-factors_mar_2018_0.pdf)

FuelCO2_FuelEU = {
    'OIL':['Crude Oil'],
    'HFO':['HFO (Residual Fuel Oil No. 6)'],
    'DSL':['Diesel Fuel (Distillate Fuel Oil No. 2)'],
    'GAS':['Natural Gas'],
    'MSW':['Municipal Solid Waste'],
    'LFG':['Landfill Gas']
}

Fuel_column = CommonParametrs_EU.columns.get_loc('Fuel')
CO2Intensity_column = CommonParametrs_EU.columns.get_loc('CO2Intensity')
Efficiency_column = CommonParametrs_EU.columns.get_loc('Efficiency')
for i, item in enumerate(CommonParametrs_EU.values):
    try:
        Fuel = FuelCO2_FuelEU[item[Fuel_column]]
        FuelRow_CO2table = CO2_emissions[CO2_emissions['Fuel']==Fuel[0]].index.values.astype(int)[0]
        item[CO2Intensity_column] = (CO2_emissions['CO2 factor (kg CO2 per mmBtu)'][FuelRow_CO2table]/1000)*(3.41214163/item[Efficiency_column])
    except:
        item[CO2Intensity_column] = 0
    CommonParametrs_EU['CO2Intensity'][i] = item[CO2Intensity_column]

c:\python27\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
CommonParametrs_EU[EU_columns]

,Technology,Fuel,Efficiency,MinUpTime,MinDownTime,RampUpRate,RampDownRate,StartUpCost,NoLoadCost,RampingCost,PartLoadMin,MinEfficiency,StartUpTime,CO2Intensity,CHPType
0,COMC,GAS,0.523159,4.615385,1.188811,0.035847,0.036577,102145.407507,340.063748,0,0.410505,0.512154,0.874126,0.346067,NaN
1,HPHS,WAT,0.865333,0.000000,0.000000,1.018893,1.018893,0.000000,0.000000,0,0.000000,0.843111,0.284444,0.000000,NaN
2,STUR,HRD,0.388409,7.787879,5.803030,0.016492,0.016735,114088.100966,68.292211,0,0.304903,0.387803,6.257576,0.000000,NaN
3,HDAM,WAT,0.883333,0.000000,0.000000,0.634963,0.634963,0.000000,0.000000,0,0.000000,0.772222,0.455556,0.000000,NaN
4,HROR,WAT,1.000000,0.000000,0.000000,1.050024,1.137491,25.120749,0.000000,0,0.000000,0.946970,0.306061,0.000000,NaN
5,PHOT,SUN,1.000000,0.000000,0.000000,0.548703,0.548703,0.000000,0.000000,0,0.000000,0.937500,0.125000,0.000000,NaN
6,WTON,WIN,1.000000,0.000000,0.000000,0.607120,0.607120,0.000000,0.000000,0,0.000000,0.958333,0.083333,0.000000,NaN
7,STUR,BIO,0.407778,5.888889,4.333333,0.014506,0.014578,7796.375710,449.237262,0,0.389523,0.396111,2.444444,0.000000,NaN
8,STUR,OIL,0.376502,7.058824,5.352941,0.022963,0.024596,29508.530032,186.160372,0,0.337731,0.362628,4.395349,0.675536,NaN
9,STUR,NUC,0.405000,24.000000,24.000000,0.003552,0.003552,200000.000000,0.000000,0,0.900000,0.405000,64.615385,0.000000,NaN


In [16]:
CommonParametrs_EU.to_csv('../Database/RawData/CommonParametrs_EU.csv', index = False)

## - Create Common Parameters (average) from SEC data

In [17]:
SaudiPowerPlants = pd.read_excel('../Database/RawData/Saudi_Gens_SEC.xlsx')

In [18]:
#Replace COA with RIYADH and remove GCC

for num, i in enumerate (SaudiPowerPlants['Node']):
    if i == 'COA':
        SaudiPowerPlants['Node'][num] ='RIYADH'
    elif i=='GCC':
        SaudiPowerPlants = SaudiPowerPlants.drop(num)
        
SaudiPowerPlants.index = range(0,len(SaudiPowerPlants))

c:\python27\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [19]:
#list all regions in the table

Regions = set(SaudiPowerPlants['Node'])
Regions = list(Regions)
Regions = [x for x in Regions if str(x) != 'nan']
Regions

[u'SOA', u'EOA', u'NEOA', 'RIYADH', u'WOA', u'NWOA', u'Hail', u'QASSIM']

In [20]:
TechnologyColumnIndex = SaudiPowerPlants.columns.get_loc("Technology")
FuelColumnIndex = SaudiPowerPlants.columns.get_loc("Fuel 1")

#Append a list of indices for columns with numbers (not string)
Integer_Indices = []
for i, ele in enumerate(range(np.shape(SaudiPowerPlants)[1])):
    try:
            np.mean(SaudiPowerPlants.values[:,ele].astype(float))
            Integer_Indices.append(ele)
    except Exception as e:
            print e.args
            pass      

AllCombs = []
for comb in SaudiPowerPlants.drop_duplicates(['Technology','Fuel 1']).values:
    Selection = np.logical_and(SaudiPowerPlants['Technology'] == comb[TechnologyColumnIndex],SaudiPowerPlants['Fuel 1'] == comb[FuelColumnIndex])
    Row = comb
    X = SaudiPowerPlants[Selection].values
    
    #Remove columns with all 'nans' because in this case np.nanmean gives error
    Integer_Indices_nonan = []
    for colum_num in Integer_Indices:
        if np.sum(np.isnan(X[:,colum_num].astype(float)) == False) > 0:
            Integer_Indices_nonan.append(colum_num)
    Row[Integer_Indices_nonan] = np.nanmean(SaudiPowerPlants[Selection].values[:,Integer_Indices_nonan],axis = 0)        
            
    AllCombs.append(Row)

('could not convert string to float: Bahra Industrial Complex',)
('could not convert string to float: CPV',)
('could not convert string to float: Solar',)
('could not convert string to float: Bahra Industrial Complex',)
('could not convert string to float: WOA',)
('could not convert string to float: IPP',)
('could not convert string to float: B',)
('could not convert string to float: E',)
('could not convert string to float: D',)
('could not convert string to float: None',)


In [21]:
'''
AllCombs = []
for comb in SaudiPowerPlants.drop_duplicates(['Technology','Fuel 1']).values[:,[TechnologyColumnIndex,FuelColumnIndex]]:
    Selection = np.logical_and(SaudiPowerPlants['Technology'] == comb[0],SaudiPowerPlants['Fuel 1'] == comb[1])
    Row = np.array(list(comb) + list(np.nanmean(SaudiPowerPlants[Selection].values[:,ToKeep].astype(float),axis=0)))
    AllCombs.append(Row)

header = SaudiPowerPlants.keys()[list([TechnologyColumnIndex,FuelColumnIndex]) + list(ToKeep)]
CommonParametrs_Saudi = pd.DataFrame(AllCombs,columns=header)
'''

"\nAllCombs = []\nfor comb in SaudiPowerPlants.drop_duplicates(['Technology','Fuel 1']).values[:,[TechnologyColumnIndex,FuelColumnIndex]]:\n    Selection = np.logical_and(SaudiPowerPlants['Technology'] == comb[0],SaudiPowerPlants['Fuel 1'] == comb[1])\n    Row = np.array(list(comb) + list(np.nanmean(SaudiPowerPlants[Selection].values[:,ToKeep].astype(float),axis=0)))\n    AllCombs.append(Row)\n\nheader = SaudiPowerPlants.keys()[list([TechnologyColumnIndex,FuelColumnIndex]) + list(ToKeep)]\nCommonParametrs_Saudi = pd.DataFrame(AllCombs,columns=header)\n"

In [22]:
SEC_columns = ['Technology','Fuel 1','Efficiency','Time Min Online','Time Min Offline','Ramp-Up (p.u. per Min)'
                       ,'Ramp-Down (p.u. per Min)','Minimum load (% of nominal capacity)']
CommonParametrs_Saudi = pd.DataFrame(AllCombs,columns=SaudiPowerPlants.keys())
CommonParametrs_Saudi[SEC_columns]

,Technology,Fuel 1,Efficiency,Time Min Online,Time Min Offline,Ramp-Up (p.u. per Min),Ramp-Down (p.u. per Min),Minimum load (% of nominal capacity)
0,GT,Gas,0.258391,11.789474,14.654971,0.097442,0.098531,0.396021
1,CC,Gas,0.597737,48.000000,48.000000,0.085000,0.085000,0.385006
2,GT,Crude,0.281724,13.147826,7.200000,0.075455,0.076833,0.436622
3,CC,Crude,0.340643,42.266667,48.000000,0.011060,0.011060,0.436750
4,GT,Diesel,0.255689,0.044118,2.352941,0.153202,0.183608,0.464635
5,ST,Gas,0.356375,48.000000,48.000000,0.023283,0.023283,0.592431
6,DE,Diesel,0.287573,0.800000,2.000000,0.242652,0.242652,0.525490
7,ST,Heavy,0.377875,48.000000,48.000000,0.016824,0.016824,0.466776
8,DE,Gas,NaN,0.000000,0.000000,0.500000,0.500000,0.000000
9,ST,Crude,0.315377,NaN,NaN,NaN,NaN,NaN


### Make some changes with assumptions

In [23]:
#Adjust DE & Gas to be similar to DE & Diesel
CommonParametrs_Saudi['Efficiency'][8] = CommonParametrs_Saudi['Efficiency'][6]
CommonParametrs_Saudi['Time Min Online'][8] = CommonParametrs_Saudi['Time Min Online'][6]
CommonParametrs_Saudi['Time Min Offline'][8] = CommonParametrs_Saudi['Time Min Offline'][6]
CommonParametrs_Saudi['Minimum load (% of nominal capacity)'][8] = CommonParametrs_Saudi['Minimum load (% of nominal capacity)'][6]

#Adjust ST & Crude to be similar to ST & Heavy
CommonParametrs_Saudi['Time Min Online'][9] = CommonParametrs_Saudi['Time Min Online'][7]
CommonParametrs_Saudi['Time Min Offline'][9] = CommonParametrs_Saudi['Time Min Offline'][7]
CommonParametrs_Saudi['Ramp-Up (p.u. per Min)'][9] = CommonParametrs_Saudi['Ramp-Up (p.u. per Min)'][7]
CommonParametrs_Saudi['Ramp-Down (p.u. per Min)'][9] = CommonParametrs_Saudi['Ramp-Down (p.u. per Min)'][7]
CommonParametrs_Saudi['Minimum load (% of nominal capacity)'][9] = CommonParametrs_Saudi['Minimum load (% of nominal capacity)'][7]

# Add other combinations and set values to be equal to somehow a similar combination
# Combination: GTUR & OIL, ICEN & OIL, STUR & OIL, STUR & BIO, STUR & WST
CommonParametrs_Saudi = CommonParametrs_Saudi.append(pd.DataFrame([['DE','Crude']+list(CommonParametrs_Saudi[SEC_columns].loc[6]['Efficiency':].values)], columns=SEC_columns),ignore_index=True)

CommonParametrs_Saudi[SEC_columns]

c:\python27\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\python27\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python27\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\python27\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

,Technology,Fuel 1,Efficiency,Time Min Online,Time Min Offline,Ramp-Up (p.u. per Min),Ramp-Down (p.u. per Min),Minimum load (% of nominal capacity)
0,GT,Gas,0.258391,11.789474,14.654971,0.097442,0.098531,0.396021
1,CC,Gas,0.597737,48.000000,48.000000,0.085000,0.085000,0.385006
2,GT,Crude,0.281724,13.147826,7.200000,0.075455,0.076833,0.436622
3,CC,Crude,0.340643,42.266667,48.000000,0.011060,0.011060,0.436750
4,GT,Diesel,0.255689,0.044118,2.352941,0.153202,0.183608,0.464635
5,ST,Gas,0.356375,48.000000,48.000000,0.023283,0.023283,0.592431
6,DE,Diesel,0.287573,0.800000,2.000000,0.242652,0.242652,0.525490
7,ST,Heavy,0.377875,48.000000,48.000000,0.016824,0.016824,0.466776
8,DE,Gas,0.287573,0.800000,2.000000,0.500000,0.500000,0.525490
9,ST,Crude,0.315377,48.000000,48.000000,0.016824,0.016824,0.466776


In [24]:
CommonParametrs_Saudi.to_csv('../Database/RawData/CommonParametrs_Saudi.csv',index = False)

## - Create Saudi power plants data, filling SEC missing data with SEC common average parameters created above

In [25]:
SaudiPowerPlants = pd.read_excel('../Database/RawData/Saudi_Gens_SEC.xlsx')
CommonParametrs_Saudi = pd.read_csv('../Database/RawData/CommonParametrs_Saudi.csv')

In [26]:
TechnologyColumnIndex = SaudiPowerPlants.columns.get_loc("Technology")
FuelColumnIndex = SaudiPowerPlants.columns.get_loc("Fuel 1")

NewSaudiPowerPlants = []
for item in SaudiPowerPlants.values:
    NewRow = []
    TechName = item[TechnologyColumnIndex]
    FuelName = item[FuelColumnIndex]
    for i,ele in enumerate(item):
        try:
            if np.isnan(float(ele)) or float(ele) == 0.0 :
                if SaudiPowerPlants.keys()[i] in CommonParametrs_Saudi.keys():
                    Selection = np.logical_and(CommonParametrs_Saudi['Technology'] ==TechName, CommonParametrs_Saudi['Fuel 1'] ==FuelName)
                    NewRow.append(np.array(CommonParametrs_Saudi[Selection][SaudiPowerPlants.keys()[i]])[0])                
                else:
                    NewRow.append(ele)
            else:
                NewRow.append(ele)
        except:
            if ele == None:
                if SaudiPowerPlants.keys()[i] in CommonParametrs_Saudi.keys():
                    Selection = np.logical_and(CommonParametrs_Saudi['Technology'] ==TechName, CommonParametrs_Saudi['Fuel 1'] ==FuelName)
                    NewRow.append(np.array(CommonParametrs_Saudi[Selection][SaudiPowerPlants.keys()[i]])[0])
            NewRow.append(ele)
    NewSaudiPowerPlants.append(NewRow)

In [27]:
#To check if the new SEC parameters data are filled correctly with the common parametrs
SEC_columns = ['Technology','Fuel 1','Efficiency','Time Min Online','Time Min Offline','Ramp-Up (p.u. per Min)'
                       ,'Ramp-Down (p.u. per Min)','Minimum load (% of nominal capacity)']
df = pd.DataFrame(NewSaudiPowerPlants,columns=SaudiPowerPlants.keys())
sample = df[df['Generator']=='Dammam GT-03']
sample[SEC_columns]

,Technology,Fuel 1,Efficiency,Time Min Online,Time Min Offline,Ramp-Up (p.u. per Min),Ramp-Down (p.u. per Min),Minimum load (% of nominal capacity)
307,GT,Gas,0.088512,11.789474,10.0,0.22462,0.22462,0.49505


#### New SEC data with missing data filled with average values of "Tech&Fuel" SEC common values

In [28]:
SEC_columns = ['Technology','Fuel 1','Efficiency','Time Min Online','Time Min Offline','Ramp-Up (p.u. per Min)'
                       ,'Ramp-Down (p.u. per Min)','Minimum load (% of nominal capacity)']
NewSaudiPowerPlants = pd.DataFrame(NewSaudiPowerPlants,columns=SaudiPowerPlants.keys())
NewSaudiPowerPlants[SEC_columns].head(5)

,Technology,Fuel 1,Efficiency,Time Min Online,Time Min Offline,Ramp-Up (p.u. per Min),Ramp-Down (p.u. per Min),Minimum load (% of nominal capacity)
0,GT,Gas,0.367361,48.0,48.0,0.10376,0.10376,0.581395
1,GT,Gas,0.367361,48.0,48.0,0.10376,0.10376,0.581395
2,GT,Gas,0.367361,48.0,48.0,0.10376,0.10376,0.581395
3,GT,Gas,0.367361,48.0,48.0,0.10376,0.10376,0.581395
4,GT,Gas,0.367361,48.0,48.0,0.10376,0.10376,0.581395


## - Writing the final version of power plants data for each Saudi region (in the required format)

In [29]:
'''
The data are constructed from SEC raw data.
1- SEC raw data are changed (above) by filling missing data with average parameters values of "Tech&Fuel" 
from common SEC parameters table. 
2- For any missing data, data from common EU parametrs table are filled in the missing data
'''

'\nThe data are constructed from SEC raw data.\n1- SEC raw data are changed (above) by filling missing data with average parameters values of "Tech&Fuel" \nfrom common SEC parameters table. \n2- For any missing data, data from common EU parametrs table are filled in the missing data\n'

In [30]:
SECkeys_toPowerPlantkeys = {
    'PowerCapacity':'Installed Capacity (MW)',
    'Unit':'Generator',
    'Zone':'Node',
    'Technology':'Technology',
    'Fuel':'Fuel 1',
    'Efficiency':'Efficiency',
    'MinUpTime':'Time Min Online',
    'MinDownTime':'Time Min Offline',
    'RampUpRate':'Ramp-Up (p.u. per Min)',
    'RampDownRate':'Ramp-Down (p.u. per Min)',
    'PartLoadMin':'Minimum load (% of nominal capacity)',
}

SECPowerPlantCommon = {
    'Installed Capacity (MW)':'PowerCapacity',
    'Generator':'Unit',
    'Node':'Zone',
    'Technology':'Technology',
    'Fuel 1':'Fuel',
    'Efficiency':'Efficiency',
    'Time Min Online':'MinUpTime',
    'Time Min Offline':'MinDownTime',
    'Ramp-Up (p.u. per Min)':'RampUpRate',
    'Ramp-Down (p.u. per Min)':'RampDownRate',
    'Minimum load (% of nominal capacity)':'PartLoadMin',
}

In [31]:
FuelTypeDict_SEC = {
    'OIL':['Crude'],
    'HFO':['Heavy'],
    'DSL':['Diesel'],
    'GAS':['Gas'],
    'SUN':['Solar'],
}
PlantTypeDict_SEC = {
    'GTUR':['GT'],
    'COMC':['CC'],
    'STUR':['ST'],
    'ICEN':['DE'],
    'PHOT':['PV'],
    'CPV':['CPV']
}

In [32]:
RegionNamesDict_SEC = {
    'SA_EOA':['EOA'],
    'SA_HAIL':['Hail'],
    'SA_NEOA':['NEOA'],
    'SA_NWOA':['NWOA'],
    'SA_QASSIM':['QASSIM'],
    'SA_RIYADH':['COA'],
    'SA_SOA':['SOA'],
    'SA_WOA':['WOA'],
}

In [33]:
Saudi_zones = ['SA_EOA','SA_HAIL','SA_NEOA','SA_NWOA','SA_QASSIM','SA_RIYADH','SA_SOA','SA_WOA']
SEC_columns = ['Installed Capacity (MW)','Generator','Node','Technology','Fuel 1','Efficiency','Time Min Online'
               ,'Time Min Offline','Ramp-Up (p.u. per Min)','Ramp-Down (p.u. per Min)','Minimum load (% of nominal capacity)']
TheHeader = ['PowerCapacity','Unit','Zone','Technology','Fuel','Efficiency','MinUpTime','MinDownTime',
             'RampUpRate','RampDownRate','StartUpCost','NoLoadCost','RampingCost','PartLoadMin',
             'MinEfficiency','StartUpTime','CO2Intensity','CHPType','CHPPowerToHeat','STOCapacity',
             'STOSelfDischarge','STOMaxChargingPower','STOChargingEfficiency']

SaudiPowerPlants={}
for reg in Saudi_zones:
    SaudiPowerPlants[reg] = pd.DataFrame(columns=TheHeader)
    Allregions = RegionNamesDict_SEC[reg]
    Filtered_SaudiPowerPlants = NewSaudiPowerPlants[SEC_columns]    #Select needed colomns
    Selection = False
    for name in Allregions:
        Selection = np.logical_or(Selection,Filtered_SaudiPowerPlants['Node'] == name)
    Selected = Filtered_SaudiPowerPlants.values[Selection]
    Selected = pd.DataFrame(Selected,columns=Filtered_SaudiPowerPlants.keys())
    for index, item in Selected.iterrows():
        MappedPlant = MapAnything(PlantTypeDict_SEC,item['Technology'])
        MappedFuel = MapAnything(FuelTypeDict_SEC,item['Fuel 1'])
        Combination = findCombinationFuelAndPlant(CommonParametrs_EU,MappedFuel,MappedPlant)
        
        if MappedPlant != '' and MappedFuel != '':
            NewRow = pd.Series(index=TheHeader)
            for Parameter in TheHeader:         
                try:
                    param = SECkeys_toPowerPlantkeys[Parameter]    #Match columns in required data format to columns in SEC data
                except:
                    param = ''
                    pass
                
                if Parameter == 'Zone':
                    NewRow[Parameter] =  reg
                elif Parameter == 'Technology':
                    NewRow[Parameter] = MappedPlant
                elif Parameter == 'Fuel':
                    NewRow[Parameter] = MappedFuel
                elif param != '' and type(item[param]) == float:
                    try:
                        float(item[param]) #use try because using "if type(item[param]) == float" accept string with underscores (e.g. SA_RIYADH is considered string)
                        if item[param]==None or np.isnan(item[param]):
                            if len(Combination)>0:
                                NewRow[Parameter] = Combination[Parameter][0]
                            else:
                                NewRow[Parameter] =  ''
                                #print 'No available EU common parameter values for combination of Fuel & Tech for: '+ MappedFuel+' & '+MappedPlant
                        else:
                            NewRow[Parameter] =  item[param]
                    except:
                        pass
                elif param == '':
                    if len(Combination)>0:
                        NewRow[Parameter] = Combination[Parameter][0]
                    else:
                        NewRow[Parameter] =  ''
                        #print 'No available EU common parameter values for combination of Fuel & Tech for: '+ MappedFuel+' & '+MappedPlant
                else:
                    NewRow[Parameter] = item[param]
        else:
            print 'Provided Technology or Fuel does not belong to the list. Technology: '+item['Technology']+' Fuel: '+item['Fuel 1']           
        SaudiPowerPlants[reg] =  SaudiPowerPlants[reg].append(NewRow,ignore_index=True)
        

In [35]:
TheHeader = [
        'PowerCapacity','Unit','Zone','Technology','Fuel','Efficiency','MinUpTime','MinDownTime',
        'RampUpRate','RampDownRate','StartUpCost','NoLoadCost','RampingCost','PartLoadMin',
        'MinEfficiency','StartUpTime','CO2Intensity','CHPType','CHPPowerToHeat','STOCapacity',
        'STOSelfDischarge','STOMaxChargingPower','STOChargingEfficiency']
for reg in Saudi_zones:   
    if not os.path.exists('../Database/PowerPlants/'+reg):
        os.makedirs('../Database/PowerPlants/'+reg)
    with open('../Database/PowerPlants/'+reg+'/2016_sto.csv','wb') as csvfile:
        SaudiPowerPlants[reg].to_csv(csvfile)
    print 'Finished '+reg
    print '====================================='

Finished SA_EOA
Finished SA_HAIL
Finished SA_NEOA
Finished SA_NWOA
Finished SA_QASSIM
Finished SA_RIYADH
Finished SA_SOA
Finished SA_WOA


## - Writing the final version of power plants data for each GCC country (in the required format)

In [36]:
'''
The data is the per technology type output of the Platts Power Plant Database. 
The data is from 2016 and features future projections until 2030. 
The files are split per technology type.
This code reads from the files and:
1- fills the missing data with average parameters values of "Tech&Fuel" from common EU parameters table.
  1.a - if non, fills the missing data with average parameters values of "Tech&Fuel" from common SEC parameters table.
2- The efficiency is changed based on values of Saudi power plants heat rates by: (Generator "Age & Type & Fuel")
'''

'\nThe data is the per technology type output of the Platts Power Plant Database. \nThe data is from 2016 and features future projections until 2030. \nThe files are split per technology type.\nThis code reads from the files and:\n1- fills the missing data with average parameters values of "Tech&Fuel" from common EU parameters table.\n  1.a - if non, fills the missing data with average parameters values of "Tech&Fuel" from common SEC parameters table.\n2- The efficiency is changed based on values of Saudi power plants heat rates by: (Generator "Age & Type & Fuel")\n'

In [37]:
GCCkeys_toPowerPlantkeys = {
    'PowerCapacity':'Capacity (MW)',
    'Unit':'Unit_Name',
    'Zone':'State_Province',
    'Technology':'Plant_Type',
    'Fuel':'Fuel_Type'}

In [38]:
SECkeys_toPowerPlantkeys = {
    'PowerCapacity':'Installed Capacity (MW)',
    'Unit':'Generator',
    'Zone':'Node',
    'Technology':'Technology',
    'Fuel':'Fuel 1',
    'Efficiency':'Efficiency',
    'MinUpTime':'Time Min Online',
    'MinDownTime':'Time Min Offline',
    'RampUpRate':'Ramp-Up (p.u. per Min)',
    'RampDownRate':'Ramp-Down (p.u. per Min)',
    'PartLoadMin':'Minimum load (% of nominal capacity)',
}

In [39]:
FuelTypeDict_GCC = {
    'WIN':['Wind'],
    'HRD':['Coal'],
    'OIL':['Oil','Crude'],
    'DSL':['Diesel'],
    'HFO':['Heavy','hfo'],
    'GAS':['Gas','Dual-Fuel'],
    'SUN':['SUN','Solar'],
    'MSW':['Industrial Waste; Municipal Solid Waste ; Wood By-Product','Municipal Solid Waste'],
    'LFG':['Landfill Gas ; Municipal Solid Waste '],
    'NUC':['Nuclear'],
    'WAT':['Hydro energy']
}
PlantTypeDict_GCC = {
    'WTON':['Onshore'],
    'GTUR':['Gas Turbine','Gas Turbine with Cogen','Bio_LG','GT'],
    'COMC':['Combined Cycle Gas Turbine (CCGT); Gas Turbine; Steam Turbine','Combined Cycle Gas Turbine (CCGT)','Integrated Gasification Combined Cycle (IGCC)','CCGT with Cogen','CC'],
    'STUR':['Steam Turbine','Steam Turbine with Cogen','PWR','Bio_MSW','ST'],
    'ICEN':['Internal Combustion','Combined Cycle Internal Combustion Plant (CCICP)','DE'],
    'HPHS':['Pumped Storage'],
    'PHOT':['PV'],
    'CSP':['CSP'],
    'CPV':['CPV'],
    'LFGG':['Bio_LG']
}

In [40]:
FuelTypeDict_GCCtoSEC = {
    'Crude':['Oil','Crude'],
    'Diesel':['Diesel'],
    'Heavy':['Heavy','hfo'],
    'Gas':['Gas','Dual-Fuel'],
    'Solar':['SUN','Solar'],
}
PlantTypeDict_GCCtoSEC = {
    'GT':['Gas Turbine','Gas Turbine with Cogen','Bio_LG','GT'],
    'CC':['Combined Cycle Gas Turbine (CCGT); Gas Turbine; Steam Turbine','Combined Cycle Gas Turbine (CCGT)','Integrated Gasification Combined Cycle (IGCC)','CCGT with Cogen','CC'],
    'ST':['Steam Turbine','Steam Turbine with Cogen','PWR','Bio_MSW','ST'],
    'DE':['Internal Combustion','Combined Cycle Internal Combustion Plant (CCICP)','DE'],
    'PV':['PV'],
    'CPV':['CPV']
}

In [41]:
FuelTypeHeatRateDict = {
    'CRUDE':['Oil'],
    'GAS':['Gas','Dual-Fuel'],
}
PlantTypeHeatRateDict = {
    'GT':['Gas Turbine','Gas Turbine with Cogen'],
    'CC':['Combined Cycle Gas Turbine (CCGT); Gas Turbine; Steam Turbine','Combined Cycle Gas Turbine (CCGT)','Integrated Gasification Combined Cycle (IGCC)','CCGT with Cogen'],
    'ST':['Steam Turbine','Steam Turbine with Cogen','PWR','Bio_MSW'],
    'DG':['Internal Combustion','Combined Cycle Internal Combustion Plant (CCICP)'],
}

In [42]:
FuelCO2_FuelEU = {
                'OIL':['Crude Oil'],
                'HFO':['HFO (Residual Fuel Oil No. 6)'],
                'DSL':['Diesel Fuel (Distillate Fuel Oil No. 2)'],
                'GAS':['Natural Gas'],
                'MSW':['Municipal Solid Waste'],
                'LFG':['Landfill Gas']}

In [43]:
RegionNamesDict_GCC = {
    'BA':['BAH','Southern','Al Muharraq','Al Hadd','Capital','Muharraq','Southern ','Capital ', 'Manama'],
    'KW':['KUW','Muhafazat al Ahmadi','Muhafazat al Wafrah','Al Jahra','Al Ahmadi','Al Ahmadi ','Al Jahra ', 'Al Farwaniyah'],
    'OM':['OMA','Al Wusta','Ruwi','Ash Sharqiyah','Ash Sharqiya','Ash Sharqia','Ash Sharqiyah North','Ash Sharqiyah South','Muscat','Musandam','Ad Dhahirah North','Al Batinah North','Batinah','North Oman','Dhofar','South Oman','Sharquiya'],
    'QA':['QAT','Al Wakrah','Al Khor','Doha'],
    'SA_EOA':['EOA','Eastern Province','Juaymah','Manifa','Shedgum'],
    'SA_HAIL':['HAIL','Hail'],
    'SA_NEOA':['NEOA','Northern Borders','Northern Borders Province','Al Jalamid','Al Jawf','Arar'],
    'SA_NWOA':['NWOA','Al Wajh','Tabuk'],
    'SA_QASSIM':['QASSIM','Al Qassim','Al Qasim'],
    'SA_RIYADH':['RIYADH','Al Riyadh','Riyadh','Wartsila'],
    'SA_SOA':['SOA','Asir','Jizan','Najran'],
    'SA_WOA':['WOA','Makkah','Al Madinah'],
    'UAE':['UAE','Sharjah','Ajman','Abu Dhabi','Dubai','Fujairah','Ras Al Khaimah','Ras al-Khaimah','Umm Al Quwain', 'Abu Dhabi ']
}

In [44]:
CommonParametrs_EU = pd.read_csv('../Database/RawData/CommonParametrs_EU.csv')
HeatRates = pd.read_csv('../Database/RawData/Heat_Rates.csv')

In [45]:
TechnologyFilesNames = ['../Database/RawData/By Technology/GCC_BioPower.xls',
                        '../Database/RawData/By Technology/GCC_Nuclear.xls',
                        '../Database/RawData/By Technology/GCC_SolarCPV.xls',
                        '../Database/RawData/By Technology/GCC_SolarCSP.xlsx',
                        '../Database/RawData/By Technology/GCC_SolarPV.xls',
                        '../Database/RawData/By Technology/GCC_Thermal.xlsx',
                        '../Database/RawData/By Technology/GCC_Wind.xls',
                  ]

In [46]:
GCC_zones = ['BA','KW','OM','QA','UAE']
TheHeader = ['PowerCapacity','Unit','Zone','Technology','Fuel','Efficiency','MinUpTime','MinDownTime',
             'RampUpRate','RampDownRate','StartUpCost','NoLoadCost','RampingCost','PartLoadMin',
             'MinEfficiency','StartUpTime','CO2Intensity','CHPType','CHPPowerToHeat','STOCapacity',
             'STOSelfDischarge','STOMaxChargingPower','STOChargingEfficiency']
TechnologyColumns_GCC = ['Capacity (MW)','Unit_Name','State_Province','Plant_Type','Fuel_Type','Year Online']

#Create one GCC power plants table from multiple raw tables
Filtered_GCCPowerPlants = pd.DataFrame(columns=TechnologyColumns_GCC)
for TechFile in TechnologyFilesNames:
        TechnologyFile = pd.read_excel(TechFile)
        TechnologyFile = TechnologyFile[TechnologyColumns_GCC]
        Filtered_GCCPowerPlants = pd.concat([Filtered_GCCPowerPlants, TechnologyFile],ignore_index=True)

GCCPowerPlants={}
for reg in GCC_zones:
    GCCPowerPlants[reg] = pd.DataFrame(columns=TheHeader)
    Allregions = RegionNamesDict_GCC[reg]
    Selection = False
    for name in Allregions:
        Selection = np.logical_or(Selection,Filtered_GCCPowerPlants['State_Province'] == name)
    Selected = Filtered_GCCPowerPlants.values[Selection]
    Selected = pd.DataFrame(Selected,columns=Filtered_GCCPowerPlants.keys())
    for index, item in Selected.iterrows():
        MappedPlant = MapAnything(PlantTypeDict_GCC,item['Plant_Type'])
        MappedFuel = MapAnything(FuelTypeDict_GCC,item['Fuel_Type'])
        MappedPlant2 = MapAnything(PlantTypeDict_GCCtoSEC,item['Plant_Type'])
        MappedFuel2 = MapAnything(FuelTypeDict_GCCtoSEC,item['Fuel_Type'])
        MappedPlant_HeatRate = MapAnything(PlantTypeHeatRateDict,item['Plant_Type'])
        MappedFuel_HeatRate = MapAnything(FuelTypeHeatRateDict,item['Fuel_Type'])
        Combination = findCombinationFuelAndPlant(CommonParametrs_EU,MappedFuel,MappedPlant)
        Combination2 = findCombinationFuelAndPlant2(CommonParametrs_Saudi,MappedFuel2,MappedPlant2)
        HeatRatesValues = findCombinationFuelAndPlant(HeatRates,MappedFuel_HeatRate,MappedPlant_HeatRate)
        
        if MappedPlant != '' and MappedFuel != '' and item['Year Online'] < 2017 and item['Year Online'] > 1950:
            NewRow = pd.Series(index=TheHeader)
            for Parameter in TheHeader:         
                try:
                    param = GCCkeys_toPowerPlantkeys[Parameter]    #Match columns in required data format to columns in GCC data
                except:
                    param = ''
                    pass
                try:
                    param2 = SECkeys_toPowerPlantkeys[Parameter]    #Match columns in required data format to columns in SEC data
                except:
                    param2 = ''
                    pass
                
                if Parameter == 'Zone':
                    NewRow[Parameter] =  reg
                elif Parameter == 'Technology':
                    NewRow[Parameter] = MappedPlant
                elif Parameter == 'Fuel':
                    NewRow[Parameter] = MappedFuel
                elif param != '' and type(item[param]) == float:
                    try:
                        float(item[param]) #use try because using "if type(item[param]) == float" accept string with underscores (e.g. SA_RIYADH is considered true)
                        if item[param]==None or np.isnan(item[param]):
                            if len(Combination)>0 and not np.isnan(Combination[Parameter][0]):
                                NewRow[Parameter] = Combination[Parameter][0]
                            elif len(Combination2)>0 and param2 != '':
                                print 'Hello'
                                NewRow[Parameter] = Combination2[param2][0]
                            else:
                                NewRow[Parameter] =  ''
                                #print 'No available EU common parameter values for combination of Fuel & Tech for: '+ MappedFuel+' & '+MappedPlant
                        else:
                            NewRow[Parameter] =  item[param]
                    except:
                        pass
                elif param == '':
                    if len(Combination)>0 and not np.isnan(Combination[Parameter][0]):
                        NewRow[Parameter] = Combination[Parameter][0]
                    elif len(Combination2)>0 and param2 != '':
                        NewRow[Parameter] = Combination2[param2][0]
                    else:
                        NewRow[Parameter] =  ''
                        #print 'No available EU common parameter values for combination of Fuel & Tech for: '+ MappedFuel+' & '+MappedPlant
                else:
                    NewRow[Parameter] = item[param]

###Change the effeciency values according to the available heat rates from Saudi power plants (By Age & Tech & Fuel)
            if len(HeatRatesValues) == 0:
                #print 'no available value for heat rate in Saudi power plants for the combination of Tech&Fuel for: '+ item['Unit_Name'],' Region: '+reg,'Fuel: '+item['Fuel_Type'],'Plant Type: '+item['Plant_Type']
                pass
            elif item['Year Online'] < 1980:
###Convert values from heat rates in "Mcal/MWh" to efficiency in "%"
                NewRow['Efficiency'] = 3412.14163/(HeatRatesValues['pre 1980s'][0] * 3.96832072)  
            elif item['Year Online'] < 1990:
                NewRow['Efficiency'] = 3412.14163/(HeatRatesValues['1980s'][0] * 3.96832072)  
            elif item['Year Online'] < 2000:
                NewRow['Efficiency'] = 3412.14163/(HeatRatesValues['1990s'][0] * 3.96832072)  
            elif item['Year Online'] < 2010:
                NewRow['Efficiency'] = 3412.14163/(HeatRatesValues['2000s'][0] * 3.96832072)  
            else:
                NewRow['Efficiency'] = 3412.14163/(HeatRatesValues['2010s'][0] * 3.96832072)

            #Changing the CO2 emissions factors according to EPA Emission Factors 
            # (https://www.epa.gov/sites/production/files/2018-03/documents/emission-factors_mar_2018_0.pdf)
            try:
                Fuel = FuelCO2_FuelEU[NewRow['Fuel']]
                FuelRow_CO2table = CO2_emissions[CO2_emissions['Fuel']==Fuel[0]].index.values.astype(int)[0]
                NewRow['CO2Intensity'] = (CO2_emissions['CO2 factor (kg CO2 per mmBtu)'][FuelRow_CO2table]/1000)*(3.41214163/NewRow['Efficiency'])
            except:
                NewRow['CO2Intensity'] = 0
    
            GCCPowerPlants[reg] =  GCCPowerPlants[reg].append(NewRow,ignore_index=True)
            
        else:
            #print 'Either: - Provided Technology or Fuel does not belong to the list. Technology: '+item['Plant_Type']+' Fuel: '+item['Fuel_Type']+ ' OR - Generator is not online yet'
            pass

$$  Heat\,Rate\,\, [\frac{MMBTU}{MWh}] = \frac{[\frac{3.41214163 MMBTU}{1 MWh}]}{Efficiency\,\, η\,\, (\%)}   $$


$$  Efficiency\,\, η\,\, (\%) = \frac{[\frac{3.41214163 MMBTU}{1 MWh}]}{Heat\,Rate\,\, [\frac{MMBTU}{MWh}]...[\frac{Mcal}{MWh} × \frac{3968.32072×10^{-6} MMBTU}{1 Mcal}]}   $$

$$ CO2\,emissions\, [\frac{Tonne\,CO2}{MWh}]  = CO2\,emissions\, [\frac{kg\,CO2}{MMBTU}] × [\frac{1\,Tonne}{1000\,kg}] ×  Heat\,Rate\,\, [\frac{MMBTU}{MWh}] $$

In [49]:
TheHeader = ['PowerCapacity','Unit','Zone','Technology','Fuel','Efficiency','MinUpTime','MinDownTime',
        'RampUpRate','RampDownRate','StartUpCost','NoLoadCost','RampingCost','PartLoadMin',
        'MinEfficiency','StartUpTime','CO2Intensity','CHPType','CHPPowerToHeat','STOCapacity',
        'STOSelfDischarge','STOMaxChargingPower','STOChargingEfficiency']

for reg in GCC_zones:   
    if not os.path.exists('../Database/PowerPlants/'+reg):
        os.makedirs('../Database/PowerPlants/'+reg)
    with open('../Database/PowerPlants/'+reg+'/2016_sto.csv','wb') as csvfile:
        GCCPowerPlants[reg].to_csv(csvfile)
    print 'Finished '+reg
    print '====================================='

Finished BA
Finished KW
Finished OM
Finished QA
Finished UAE


### * Read power plant files from separate folders inside 'PowerPlants' folder and write them in one file as 'Units_testcase.csv' in 'DummyData_all' folder

In [14]:
zones = ['../Database/PowerPlants/SA_EOA/2016_sto.csv',
         '../Database/PowerPlants/SA_HAIL/2016_sto.csv',
         '../Database/PowerPlants/SA_NEOA/2016_sto.csv',
         '../Database/PowerPlants/SA_NWOA/2016_sto.csv',
         '../Database/PowerPlants/SA_QASSIM/2016_sto.csv',
         '../Database/PowerPlants/SA_RIYADH/2016_sto.csv',
         '../Database/PowerPlants/SA_SOA/2016_sto.csv',
         '../Database/PowerPlants/SA_WOA/2016_sto.csv',
         '../Database/PowerPlants/BA/2016_sto.csv',
         '../Database/PowerPlants/KW/2016_sto.csv',
         '../Database/PowerPlants/OM/2016_sto.csv',
         '../Database/PowerPlants/QA/2016_sto.csv',
         '../Database/PowerPlants/UAE/2016_sto.csv',
                  ]

In [15]:
AllPowerPlants = pd.read_csv(zones[0])
for zone in zones[1:]:
        ZonePowerPlants = pd.read_csv(zone)
        AllPowerPlants = pd.concat([AllPowerPlants, ZonePowerPlants],ignore_index=True)

AllPowerPlants = AllPowerPlants.iloc[:,1:].copy()

In [52]:
AllPowerPlants.to_csv('../Database/DummyData_all/Units_testcase.csv', index = False)

# 3- Fuel prices data

### Random snippets

In [6]:
AllPowerPlants

,PowerCapacity,Unit,Zone,Technology,Fuel,Efficiency,MinUpTime,MinDownTime,RampUpRate,RampDownRate,...,PartLoadMin,MinEfficiency,StartUpTime,CO2Intensity,CHPType,CHPPowerToHeat,STOCapacity,STOSelfDischarge,STOMaxChargingPower,STOChargingEfficiency
0,430.00,Ghazlan GT-01,SA_EOA,STUR,GAS,0.320300,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
1,430.00,Ghazlan GT-02,SA_EOA,STUR,GAS,0.313903,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
2,430.00,Ghazlan GT-03,SA_EOA,STUR,GAS,0.318038,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
3,430.00,Ghazlan GT-04,SA_EOA,STUR,GAS,0.315202,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
4,664.00,Ghazlan GT-05,SA_EOA,STUR,GAS,0.378654,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
5,664.00,Ghazlan GT-06,SA_EOA,STUR,GAS,0.378441,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
6,664.00,Ghazlan GT-07,SA_EOA,STUR,GAS,0.379327,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
7,664.00,Ghazlan GT-08,SA_EOA,STUR,GAS,0.381263,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
8,625.00,Qurrayah ST-01,SA_EOA,STUR,GAS,0.375210,48.000000,48.000000,0.024000,0.024000,...,0.640000,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN
9,625.00,Qurrayah ST-02,SA_EOA,STUR,GAS,0.375685,48.000000,48.000000,0.024000,0.024000,...,0.640000,0.312000,4.395349,0.535646,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
import sys
sys.path.append("..")
import DispaSET as ds
from DispaSET.postprocessing import postprocessing as post
reload(post)
from DispaSET.preprocessing import utils as utils
reload(utils)
from DispaSET.preprocessing import preprocessing as prep
reload(prep)

<module 'DispaSET.preprocessing.preprocessing' from '..\DispaSET\preprocessing\preprocessing.pyc'>

In [9]:
Plants_merged, mapping = utils.clustering(AllPowerPlants, method='LP')

[INFO    ] (clustering): Clustered 1018 original units into 67 new units


In [20]:
Plants_merged

,PowerCapacity,Unit,Zone,Technology,Fuel,Efficiency,MinUpTime,MinDownTime,RampUpRate,RampDownRate,...,PartLoadMin,MinEfficiency,StartUpTime,CO2Intensity,CHPType,CHPPowerToHeat,STOCapacity,STOSelfDischarge,STOMaxChargingPower,STOChargingEfficiency
Unit,,,,,,,,,,,,,,,,,,,,,
"[0L, 1L, 2L, 3L, 4L, ... _EOA - STUR - GAS -",6876.000000,"[0L, 1L, 2L, 3L, 4L, ... _EOA - STUR - GAS -",SA_EOA,STUR,GAS,0.361380,48.000000,48.000000,0.023270,0.023270,...,0.033450,0.312000,0.274869,0.535646,,NaN,NaN,NaN,NaN,NaN
"[12L, 13L, 14L, 15L, ... _EOA - GTUR - GAS -",5700.153050,"[12L, 13L, 14L, 15L, ... _EOA - GTUR - GAS -",SA_EOA,GTUR,GAS,0.262163,27.226178,22.946459,0.132940,0.132940,...,0.000877,0.255900,0.000747,0.500133,,NaN,NaN,NaN,NaN,NaN
"[30L, 31L, 32L, 33L, 34L, 35L] - SA_EOA - COMC - GAS -",3990.000000,"[30L, 31L, 32L, 33L, 34L, 35L] - SA_EOA - COMC...",SA_EOA,COMC,GAS,0.454413,48.000000,48.000000,0.100000,0.100000,...,0.048120,0.512154,0.145688,0.346067,,NaN,NaN,NaN,NaN,NaN
"[58L, 59L, 60L, 61L, 62L, 63L] - SA_EOA - GTUR - DSL -",121.600000,"[58L, 59L, 60L, 61L, 62L, 63L] - SA_EOA - GTUR...",SA_EOA,GTUR,DSL,0.181398,0.044118,10.000000,0.092378,0.092378,...,0.041118,0.169000,0.083882,0.870214,,NaN,NaN,NaN,NaN,NaN
"[105L, 106L, 107L, 1 ... _EOA - ICEN - GAS -",11700.000000,"[105L, 106L, 107L, 1 ... _EOA - ICEN - GAS -",SA_EOA,ICEN,GAS,0.287573,0.800000,2.000000,0.500000,0.500000,...,0.006737,0.406300,0.003846,0.430043,,NaN,NaN,NaN,NaN,NaN
[113L] - Saudi Aramco North Park Project,10.500000,[113L] - Saudi Aramco North Park Project,SA_EOA,PHOT,SUN,1.000000,0.000000,0.000000,0.548703,0.548703,...,0.000000,0.937500,0.125000,0.000000,,NaN,NaN,NaN,NaN,NaN
"[114L, 115L, 116L, 1 ... HAIL - GTUR - OIL -",850.608500,"[114L, 115L, 116L, 1 ... HAIL - GTUR - OIL -",SA_HAIL,GTUR,OIL,0.284250,4.000000,2.000000,0.069089,0.069089,...,0.029391,0.169000,0.043018,0.877038,,NaN,NaN,NaN,NaN,NaN
"[126L, 127L, 128L, 1 ... NEOA - GTUR - OIL -",343.000000,"[126L, 127L, 128L, 1 ... NEOA - GTUR - OIL -",SA_NEOA,GTUR,OIL,0.246478,13.147826,2.000000,0.113701,0.113701,...,0.034985,0.169000,0.040233,0.877038,,NaN,NaN,NaN,NaN,NaN
"[136L, 137L, 138L, 1 ... NEOA - GTUR - DSL -",369.000000,"[136L, 137L, 138L, 1 ... NEOA - GTUR - DSL -",SA_NEOA,GTUR,DSL,0.263891,0.044118,2.000000,0.592140,0.846883,...,0.021680,0.169000,0.024390,0.870214,,NaN,NaN,NaN,NaN,NaN


In [11]:
Plants_merged2, mapping2 = utils.clustering(AllPowerPlants, method='Standard', Nslices=3, PartLoadMax=0.1, Pmax=30)

[INFO    ] (clustering): Clustered 1018 original units into 776 new units


In [19]:
Plants_merged2

,PowerCapacity,Unit,Zone,Technology,Fuel,Efficiency,MinUpTime,MinDownTime,RampUpRate,RampDownRate,...,PartLoadMin,MinEfficiency,StartUpTime,CO2Intensity,CHPType,CHPPowerToHeat,STOCapacity,STOSelfDischarge,STOMaxChargingPower,STOChargingEfficiency
Unit,,,,,,,,,,,,,,,,,,,,,
[0L] - Ghazlan GT-01,430.00,[0L] - Ghazlan GT-01,SA_EOA,STUR,GAS,0.320300,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[1L] - Ghazlan GT-02,430.00,[1L] - Ghazlan GT-02,SA_EOA,STUR,GAS,0.313903,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[2L] - Ghazlan GT-03,430.00,[2L] - Ghazlan GT-03,SA_EOA,STUR,GAS,0.318038,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[3L] - Ghazlan GT-04,430.00,[3L] - Ghazlan GT-04,SA_EOA,STUR,GAS,0.315202,48.000000,48.000000,0.023260,0.023260,...,0.534884,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[4L] - Ghazlan GT-05,664.00,[4L] - Ghazlan GT-05,SA_EOA,STUR,GAS,0.378654,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[5L] - Ghazlan GT-06,664.00,[5L] - Ghazlan GT-06,SA_EOA,STUR,GAS,0.378441,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[6L] - Ghazlan GT-07,664.00,[6L] - Ghazlan GT-07,SA_EOA,STUR,GAS,0.379327,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[7L] - Ghazlan GT-08,664.00,[7L] - Ghazlan GT-08,SA_EOA,STUR,GAS,0.381263,48.000000,48.000000,0.022590,0.022590,...,0.602410,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN
[8L] - Qurrayah ST-01,625.00,[8L] - Qurrayah ST-01,SA_EOA,STUR,GAS,0.375210,48.000000,48.000000,0.024000,0.024000,...,0.640000,0.312000,4.395349,0.535646,,NaN,NaN,NaN,NaN,NaN


In [ ]:
tmp = pd.DataFrame(mapping)
tmp['FormerIndexes'].dropna()

In [24]:
Plants_merged3, mapping3 = utils.clustering(AllPowerPlants, method='Standard', subMethod= 'TechFuelCluster')

[INFO    ] (clustering): Clustered 1018 original units into 67 new units


In [17]:
Plants_merged4, mapping4 = utils.clustering(AllPowerPlants, method='Standard', subMethod= 'FuelCluster')

[INFO    ] (clustering): Clustered 1018 original units into 41 new units


In [18]:
Plants_merged4

,PowerCapacity,Unit,Zone,Technology,Fuel,Efficiency,MinUpTime,MinDownTime,RampUpRate,RampDownRate,...,PartLoadMin,MinEfficiency,StartUpTime,CO2Intensity,CHPType,CHPPowerToHeat,STOCapacity,STOSelfDischarge,STOMaxChargingPower,STOChargingEfficiency
Unit,,,,,,,,,,,,,,,,,,,,,
"[0L, 1L, 2L, 3L, 4L, ... _EOA - STUR - GAS -",28266.153050,"[0L, 1L, 2L, 3L, 4L, ... _EOA - STUR - GAS -",SA_EOA,STUR,GAS,0.323954,24.273603,23.907262,0.253546,0.253546,...,0.000177,0.367973,1.507103e-04,0.458012,,NaN,NaN,NaN,NaN,NaN
"[58L, 59L, 60L, 61L, 62L, 63L] - SA_EOA - GTUR - DSL -",121.600000,"[58L, 59L, 60L, 61L, 62L, 63L] - SA_EOA - GTUR...",SA_EOA,GTUR,DSL,0.181398,0.044118,10.000000,0.092378,0.092378,...,0.041118,0.169000,8.388158e-02,0.870214,,NaN,NaN,NaN,NaN,NaN
[113L] - Saudi Aramco North Park Project,10.500000,[113L] - Saudi Aramco North Park Project,SA_EOA,PHOT,SUN,1.000000,0.000000,0.000000,0.548703,0.548703,...,0.000000,0.937500,1.250000e-01,0.000000,,NaN,NaN,NaN,NaN,NaN
"[114L, 115L, 116L, 1 ... HAIL - GTUR - OIL -",850.608500,"[114L, 115L, 116L, 1 ... HAIL - GTUR - OIL -",SA_HAIL,GTUR,OIL,0.284250,4.000000,2.000000,0.069089,0.069089,...,0.029391,0.169000,4.301779e-02,0.877038,,NaN,NaN,NaN,NaN,NaN
"[126L, 127L, 128L, 1 ... NEOA - GTUR - OIL -",343.000000,"[126L, 127L, 128L, 1 ... NEOA - GTUR - OIL -",SA_NEOA,GTUR,OIL,0.246478,13.147826,2.000000,0.113701,0.113701,...,0.034985,0.169000,4.023324e-02,0.877038,,NaN,NaN,NaN,NaN,NaN
"[136L, 137L, 138L, 1 ... NEOA - GTUR - DSL -",369.000000,"[136L, 137L, 138L, 1 ... NEOA - GTUR - DSL -",SA_NEOA,GTUR,DSL,0.263891,0.044118,2.000000,0.592140,0.846883,...,0.021680,0.169000,2.439024e-02,0.870214,,NaN,NaN,NaN,NaN,NaN
"[146L, 147L, 151L, 152L] - SA_NWOA - GTUR - DSL -",668.200000,"[146L, 147L, 151L, 152L] - SA_NWOA - GTUR - DSL -",SA_NWOA,GTUR,DSL,0.240773,0.116742,2.319031,0.161797,0.189281,...,0.023593,0.191800,1.346902e-02,0.844198,,NaN,NaN,NaN,NaN,NaN
"[148L, 149L] - SA_NWOA - STUR - OIL -",1962.000000,"[148L, 149L] - SA_NWOA - STUR - OIL -",SA_NWOA,STUR,OIL,0.310459,45.122298,44.631193,0.021665,0.021779,...,0.036051,0.346640,4.954128e-02,0.692174,,NaN,NaN,NaN,NaN,NaN
[150L] - Saudi Aramco Soitec Solar CPV Park,1.100000,[150L] - Saudi Aramco Soitec Solar CPV Park,SA_NWOA,CPV,SUN,1.000000,0.000000,0.000000,0.548703,0.548703,...,0.000000,0.937500,1.250000e-01,0.000000,,NaN,NaN,NaN,NaN,NaN


In [14]:
np.linspace(0, 1, 13)

array([0.        , 0.08333333, 0.16666667, 0.25      , 0.33333333,
       0.41666667, 0.5       , 0.58333333, 0.66666667, 0.75      ,
       0.83333333, 0.91666667, 1.        ])

In [15]:
AAAAA = pd.DataFrame(columns=AllPowerPlants.columns)
for i in AAAAA.index:
    print i

In [16]:
AABB = [1, 2, 3, 4]
AACC = [4, 3, 2, 4]
all(AABB[i] == AACC[i] for i in range(5))

False